# Nigep example using MNIST dataset

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/*'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.nigep import Nigep
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.optimizers.legacy import RMSprop
from keras.datasets import mnist
from keras.utils import to_categorical


### Load and preprocess MNIST dataset from Keras package

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train / 255
X_test = X_test / 255

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])

INPUT_DIMS = X_train.shape[1]
OUTPUT_DIMS = 10

y_train_one_hot = to_categorical(y_train, num_classes=OUTPUT_DIMS)

### Build and compile the MLP model

In [3]:
NUM_EPOCHS = 20

model = Sequential()

model.add(Dense(512, activation="sigmoid", input_shape=(INPUT_DIMS,), name="Hidden_layer_1"))
model.add(Dense(128, activation="sigmoid", name="Hidden_layer_2"))

model.add(Dense(OUTPUT_DIMS, activation="softmax", name="output_layer"))

model.build()

model.compile(optimizer=RMSprop(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden_layer_1 (Dense)      (None, 512)               401920    
                                                                 
 Hidden_layer_2 (Dense)      (None, 128)               65664     
                                                                 
 output_layer (Dense)        (None, 10)                1290      
                                                                 
Total params: 468874 (1.79 MB)
Trainable params: 468874 (1.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Execute Nigep to obtain the Generalization Profile results

In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.001, patience=5, verbose=1)
erl_stopping = EarlyStopping(patience=4, monitor='val_loss', verbose=1)
callbacks = [erl_stopping, lr_reduce]

nigep = Nigep(
    execution_name='mnist',
    x_data=X_train,
    y_data=y_train_one_hot,
    model=model,
    batch_size=128,
    input_shape=INPUT_DIMS,
    class_mode='categorical',
    k_fold_n=5,
    target_names=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
    epochs=20,
)

nigep.fit()

Epoch 1/20
Epoch 1/20
1350/1350 [==============================] - 4s 3ms/step - loss: 0.3964 - accuracy: 0.8860 - val_loss: 0.2039 - val_accuracy: 0.9406
Epoch 2/20
Epoch 2/20
1350/1350 [==============================] - 5s 3ms/step - loss: 0.1849 - accuracy: 0.9443 - val_loss: 0.1528 - val_accuracy: 0.9545
Epoch 3/20
Epoch 3/20
1350/1350 [==============================] - 5s 4ms/step - loss: 0.1379 - accuracy: 0.9596 - val_loss: 0.1090 - val_accuracy: 0.9667
Epoch 4/20
1350/1350 [==============================] - 5s 4ms/step - loss: 0.1319 - accuracy: 0.9604 - val_loss: 0.1085 - val_accuracy: 0.9667
Epoch 4/20
1350/1350 [==============================] - 5s 3ms/step - loss: 0.1295 - accuracy: 0.9639 - val_loss: 0.1087 - val_accuracy: 0.9773
Epoch 5/20
Epoch 5/20
1350/1350 [==============================] - 5s 3ms/step - loss: 0.0981 - accuracy: 0.9719 - val_loss: 0.1057 - val_accuracy: 0.9700
Epoch 6/20
Epoch 6/20
1350/1350 [==============================] - 4s 3ms/step - loss: 0.079

/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

 90/375 [======>.......................] - ETA: 0s

/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/vmberto/Documents/nigep/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

375/375 [==============================] - 0s 1ms/step
Epoch 1/20
 232/1350 [====>.........................] - ETA: 3s - loss: 0.9883 - accuracy: 0.6644

In [ ]:
nigep.plot_and_save_generalization_profile()